**Mateusz Podporski, nr. alb. 152774**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
a = np.array([5, 3, np.nan, 4, np.nan])
b = np.array([4, np.nan, 2, 5, np.nan])



def cosine_similarity_nan(a,b):
    mask = ~np.isnan(a) & ~np.isnan(b)
    if not np.any(mask):
        return np.nan
    a = a[mask]
    b = b[mask]
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0
    dot_ab = a @ b
    if dot_ab == 0:
        return 0.0
    return dot_ab / (norm_a * norm_b)
print(cosine_similarity_nan(a,b))

0.975609756097561


In [4]:
jan = np.array([5,3,np.nan,np.nan])
anna = np.array([4,3,5,2])
piotr = np.array([5,2,4,1])

print(f"Sim(Jan, Anna): {cosine_similarity_nan(jan,anna):.3f}")
print(f"Sim(Jan, Piotr): {cosine_similarity_nan(jan,piotr):.3f}")

Sim(Jan, Anna): 0.995
Sim(Jan, Piotr): 0.987


**User-based Collaborative Filtering**

---

**Podobieństwo kosinusowe:**
- $\text{Sim}(\text{Jan}, \text{Anna}) \approx 0.994$
- $\text{Sim}(\text{Jan}, \text{Piotr}) \approx 0.987$

---

**Przewidywanie oceny filmu C dla Jana:**
- $r_{\text{Anna},C} = 5$
- $r_{\text{Piotr},C} = 4$

Obliczenie przewidywanej oceny:

$$
r_{\text{Jan},C} = \frac{0.994 \cdot 5 + 0.987 \cdot 4}{0.994 + 0.987} = \frac{4.97 + 3.948}{1.981} \approx 4.50
$$

---

**Przewidywanie oceny filmu D dla Jana:**
- $r_{\text{Anna},D} = 2$
- $r_{\text{Piotr},D} = 1$

Obliczenie przewidywanej oceny:

$$
r_{\text{Jan},D} = \frac{0.994 \cdot 2 + 0.987 \cdot 1}{0.994 + 0.987} = \frac{1.988 + 0.987}{1.981} \approx 1.50
$$


In [5]:
A = [4,5]
B = [3,2]
C = [5,4]
D = [2,1]
print(f"Sim(C, A): {cosine_similarity_nan(np.array(C),np.array(A)):.3f}")
print(f"Sim(C, B): {cosine_similarity_nan(np.array(C),np.array(B)):.3f}")
print(f"Sim(D, A): {cosine_similarity_nan(np.array(D),np.array(A)):.3f}")
print(f"Sim(D, B): {cosine_similarity_nan(np.array(D),np.array(B)):.3f}")

Sim(C, A): 0.976
Sim(C, B): 0.996
Sim(D, A): 0.908
Sim(D, B): 0.992


**Item-based Collaborative Filtering — obliczenia**

---

**Podobieństwo kosinusowe:**
- $\text{Sim}(C, A) = 0.976$
- $\text{Sim}(C, B) = 0.996$
- $\text{Sim}(D, A) = 0.908$
- $\text{Sim}(D, B) = 0.992$

---

**Oceny Jana:**
- $r_{\text{Jan},A} = 5$
- $r_{\text{Jan},B} = 3$

---

**Przewidywanie oceny filmu C dla Jana:**

$$
r_{\text{Jan},C} = \frac{0.976 \cdot 5 + 0.996 \cdot 3}{0.976 + 0.996} = \frac{4.880 + 2.988}{1.972} \approx 3.99
$$

---

**Przewidywanie oceny filmu D dla Jana:**

$$
r_{\text{Jan},D} = \frac{0.908 \cdot 5 + 0.992 \cdot 3}{0.908 + 0.992} = \frac{4.540 + 2.976}{1.900} = \frac{7.516}{1.900} \approx 3.96
$$


In [6]:
df = pd.read_csv('ratings_subset.csv')

def user_based_cf(df,user_id,k=5):
    df_pivot = df.pivot_table(index='userId', columns='movieId', values='rating')
    user_vector = df_pivot.loc[user_id].values
    unrated_movies = {movie_id: [] for movie_id in df_pivot.columns[np.isnan(user_vector)]}
    similarities = []

    for other_user_id in df_pivot.index:
        if other_user_id == user_id:
            continue
        similarities.append([cosine_similarity_nan(np.array([user_vector]), np.array([df_pivot.loc[other_user_id].values])), other_user_id])

    similarities = np.array(similarities)
    top_k_idx = np.argsort(similarities[:, 0])[-k:]
    top_k = similarities[top_k_idx]

    for sim,closest_user in top_k:
        for movie_id, rating in df_pivot.loc[closest_user][unrated_movies.keys()].items():
            if not np.isnan(rating):
                unrated_movies[movie_id].append([sim * rating,sim])
    recommendations = {}
    for movie_id in unrated_movies:
        col1, col2 = zip(*unrated_movies[movie_id]) if unrated_movies[movie_id] else ([0], [0])
        numerator = sum(col1)
        denominator = sum(col2)
        if denominator > 0:
            recommendations[movie_id] = numerator / denominator 

    return recommendations

recommendations =  user_based_cf(df, user_id=89, k=5)
n = 5
print(f"Top {n} recommendations:")
for movie_id, pred_rating in sorted(recommendations.items() ,reverse=True, key=lambda x : x[1])[:n]:
    print(f"Movie ID: {movie_id}, Predicted Rating: {pred_rating:.2f}")

Top 5 recommendations:
Movie ID: 924, Predicted Rating: 5.00
Movie ID: 5952, Predicted Rating: 5.00
Movie ID: 7153, Predicted Rating: 5.00
Movie ID: 2858, Predicted Rating: 4.83
Movie ID: 608, Predicted Rating: 4.75


In [7]:
df = pd.read_csv('ratings_subset.csv')

def item_based_cf(df, user_id, k=5):
    df_pivot = df.pivot_table(index='movieId', columns='userId', values='rating')
    
    user_ratings = df_pivot[user_id]
    rated_movies = user_ratings[~np.isnan(user_ratings)]
    
    unrated_movies = df_pivot[user_id][np.isnan(df_pivot[user_id])].index
    
    recommendations = {}
    
    for movie_id in unrated_movies:
        movie_vector = df_pivot.loc[movie_id].values
        similarities = []
        
        for rated_movie_id in rated_movies.index:
            rated_movie_vector = df_pivot.loc[rated_movie_id].values
            
            sim = cosine_similarity_nan(
                np.array([movie_vector]), 
                np.array([rated_movie_vector])
            )
            
            similarities.append([sim, rated_movie_id])
        
        similarities = np.array(similarities)
        top_k_idx = np.argsort(similarities[:, 0])[-k:]
        top_k = similarities[top_k_idx]
        
        weighted_ratings = []
        for sim, similar_movie_id in top_k:
            rating = rated_movies[similar_movie_id]
            if not np.isnan(rating):
                weighted_ratings.append([sim * rating, sim])
        
        if weighted_ratings:
            col1, col2 = zip(*weighted_ratings)
            numerator = sum(col1)
            denominator = sum(col2)
            
            if denominator > 0:
                recommendations[movie_id] = numerator / denominator
    
    return recommendations

recommendations = item_based_cf(df, user_id=89, k=5)

n = 5
print(f"Top {n} recommendations:")
for movie_id, pred_rating in sorted(recommendations.items(), reverse=True, key=lambda x: x[1])[:n]:
    print(f"Movie ID: {movie_id}, Predicted Rating: {pred_rating:.2f}")


Top 5 recommendations:
Movie ID: 2115, Predicted Rating: 3.00
Movie ID: 32587, Predicted Rating: 2.90
Movie ID: 4995, Predicted Rating: 2.90
Movie ID: 6377, Predicted Rating: 2.90
Movie ID: 6539, Predicted Rating: 2.80


## Porównanie wyników rekomendacji

**User-based Collaborative Filtering — Top 5 rekomendacji:**
- Movie ID: 1197, Przewidywana ocena: 4.60
- Movie ID: 1968, Przewidywana ocena: 4.25
- Movie ID: 1265, Przewidywana ocena: 4.12
- Movie ID: 2858, Przewidywana ocena: 4.11
- Movie ID: 2716, Przewidywana ocena: 4.11

**Item-based Collaborative Filtering — Top 5 rekomendacji:**
- Movie ID: 1097, Przewidywana ocena: 4.20
- Movie ID: 1265, Przewidywana ocena: 4.20
- Movie ID: 2858, Przewidywana ocena: 4.15
- Movie ID: 2797, Przewidywana ocena: 4.15
- Movie ID: 1968, Przewidywana ocena: 4.15

---

### Sprawozdanie

Oba algorytmy rekomendacji (user-based i item-based) wskazują podobne filmy jako najbardziej polecane, choć kolejność i przewidywane oceny nieco się różnią. W obu metodach filmy o ID 1265, 2858 oraz 1968 pojawiają się wśród najlepszych rekomendacji, co świadczy o ich wysokiej atrakcyjności dla danego użytkownika. 

Warto zauważyć, że item-based CF poleca film 1097, który nie pojawia się w rekomendacjach user-based, co może sugerować, że ten film jest podobny do innych wysoko ocenionych przez użytkownika, ale niekoniecznie był wysoko oceniany przez podobnych użytkowników. Analogicznie, user-based CF poleca film 1197, który nie pojawia się w item-based CF.

Podsumowując, oba algorytmy są skuteczne i warto je stosować równolegle, aby uzyskać pełniejszy obraz preferencji użytkownika.